In [4]:
from SagasuSubs.database import FileCrud,DialogCrud,dto,SeriesCrud,EpisodeCrud
from SagasuSubs.api.utils import NotebookAPIUtils

from pathlib import Path

In [5]:
DB_PATH = Path('.') / 'db.sqlite3'

file_crud = FileCrud(DB_PATH)
dialog_crud =DialogCrud(DB_PATH)
series_crud = SeriesCrud(DB_PATH)
episode_crud = EpisodeCrud(DB_PATH)

In [6]:
api = NotebookAPIUtils('http://localhost:3000')

api.search('魔法少女小圆'),api.episodes(11017)

(SeriesCreate(id=11017, name='魔法少女まどか☆マギカ'),
 [EpisodeCreate(id=131114, name='夢の中で会った、ような……', sort=1.0, type=0, series_id=11017),
  EpisodeCreate(id=131115, name='コネクト', sort=1.0, type=2, series_id=11017),
  EpisodeCreate(id=131116, name='Magia', sort=1.0, type=3, series_id=11017),
  EpisodeCreate(id=131117, name='それはとっても嬉しいなって', sort=2.0, type=0, series_id=11017),
  EpisodeCreate(id=131118, name='もう何も恐くない', sort=3.0, type=0, series_id=11017),
  EpisodeCreate(id=131119, name='奇跡も、魔法も、あるんだよ', sort=4.0, type=0, series_id=11017),
  EpisodeCreate(id=131120, name='後悔なんて あるわけない', sort=5.0, type=0, series_id=11017),
  EpisodeCreate(id=131121, name='こんなの絶対おかしいよ', sort=6.0, type=0, series_id=11017),
  EpisodeCreate(id=131122, name='本当の気持ちと向き合えますか？', sort=7.0, type=0, series_id=11017),
  EpisodeCreate(id=131123, name='あたしって、ほんとバカ', sort=8.0, type=0, series_id=11017),
  EpisodeCreate(id=131124, name='そんなの、あたしが許さない', sort=9.0, type=0, series_id=11017),
  EpisodeCreate(id=131125, name='もう誰にも頼らない', 

In [7]:
try:
    series_crud.create_table()
    episode_crud.create_table()
except Exception as e:
    print(e)

(sqlite3.OperationalError) table series already exists
[SQL: 
CREATE TABLE series (
	id INTEGER NOT NULL, 
	name VARCHAR NOT NULL, 
	PRIMARY KEY (id)
)

]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


In [8]:
from tqdm.notebook import tqdm
from typing import Dict

files = tqdm(file_crud)

for count,file in enumerate(files):
    file:dto.FileRead
    if not file.series_name:
        continue

    series = api.search(file.series_name) 
    if not series:
        continue

    if not series_crud.read(series.id):
        series_crud.create(series)

    for episode in api.episodes(series.id):
        if not episode_crud.read(episode.id):
            episode_crud.create(episode)

    file.series_id = series.id
    file_crud.update(file)
    
    files.set_description(f'{file.id=}, {file.series_name}, {series.name}')


  0%|          | 0/41103 [00:00<?, ?it/s]